In [1]:
import tensorflow as tf

C:\Users\Bing\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Bing\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Bing\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Bing\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
from tensorflow import keras

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [3]:
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test)= fashion_mnist.load_data()

In [4]:
X_train_full.shape

(60000, 28, 28)

In [5]:
y_train_full.shape

(60000,)

In [6]:
X_test.shape

(10000, 28, 28)

In [7]:
y_test.shape

(10000,)

In [8]:
# getting max value for scaling purpose
print(X_train_full.max())
print(X_test.max())

255
255


In [9]:
# split train_full to train and valid set  
# scaling input features down to range 0-1 by dividing them by 255.0 (also converts to float)

X_valid, X_train= X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train= y_train_full[:5000], y_train_full[5000:]

In [10]:
X_train.shape

(55000, 28, 28)

In [11]:
X_valid.shape

(5000, 28, 28)

In [12]:
y_valid.shape

(5000,)

In [13]:
X_train= X_train.reshape(55000, 784)
X_valid= X_valid.reshape(5000, 784)
X_test= X_test.reshape(10000, 784)

In [14]:
# creating class name for identifying output
class_names= ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
 "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [15]:
class_names[y_train[0]]

'Coat'

In [16]:
# Building model with functional API

input= keras.layers.Input(shape=(784,))
hidden1= keras.layers.Dense(300, activation='relu')(input)
hidden2= keras.layers.Dense(300, activation='relu')(hidden1)
concat= keras.layers.Concatenate()([input, hidden2])
output= keras.layers.Dense(10, activation= 'softmax')(concat)
model= keras.models.Model(inputs=[input], outputs=[output])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 300)          235500      input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          90300       dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 1084)         0           input_1[0][0]                    
                                                                 dense_1[0][0]                

In [18]:
keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [19]:
# compiling the model
model.compile(loss='sparse_categorical_crossentropy', 
             optimizer= 'sgd', metrics=['accuracy'])

In [20]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb= keras.callbacks.ModelCheckpoint('fashion_mnist_model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [21]:
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
 6752/55000 [==>...........................] - ETA: 1:02 - loss: 1.3466 - acc: 0.6013

KeyboardInterrupt: 

In [23]:
model = keras.models.load_model("fashion_mnist_model.h5") # rollback to best model
model.evaluate(X_test, y_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
10000/10000 [==============================] - 3s 347us/sample - loss: 65.5108 - acc: 0.8524


[65.51082185438871, 0.8524]

## hyperparameter tuning for learning rate

In [24]:
K = keras.backend

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_batch_end(self, batch, logs):
        self.rates.append(K.get_value(self.model.optimizer.lr))
        self.losses.append(logs["loss"])
        K.set_value(self.model.optimizer.lr, self.model.optimizer.lr * self.factor)

In [26]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

In [27]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])
expon_lr = ExponentialLearningRate(factor=1.005)

ValueError: Tensor("dense_2_1/Softmax:0", shape=(?, 10), dtype=float32) must be from the same graph as Tensor("dense_2_target:0", shape=(?, ?), dtype=float32).

In [ ]:
history = model.fit(X_train, y_train, epochs=1,
                    validation_data=(X_valid, y_valid),
                    callbacks=[expon_lr])

In [28]:
plt.plot(expon_lr.rates, expon_lr.losses)
plt.gca().set_xscale('log')
plt.hlines(min(expon_lr.losses), min(expon_lr.rates), max(expon_lr.rates))
plt.axis([min(expon_lr.rates), max(expon_lr.rates), 0, expon_lr.losses[0]])
plt.xlabel("Learning rate")
plt.ylabel("Loss")

NameError: name 'expon_lr' is not defined

The loss starts shooting back up violently around 3e-1, so let's try using 2e-1 as our learning rate:

In [29]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

AttributeError: module 'tensorflow._api.v1.random' has no attribute 'set_seed'

## new model (1)

In [30]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [31]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=2e-1),
              metrics=["accuracy"])

In [32]:
checkpoint_cb= keras.callbacks.ModelCheckpoint('1fashion_mnist_model.h5', save_best_only=True)

In [33]:
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

ValueError: Error when checking input: expected flatten_input to have 3 dimensions, but got array with shape (55000, 784)

In [34]:
new_model1 = keras.models.load_model("1mnist_model.h5") # rollback to best model
new_model1.evaluate(X_test, y_test)

OSError: Unable to open file (unable to open file: name = '1mnist_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## tuning using scikit learn searchCV

In [35]:
#scikit learn sequential
def build_model(n_hidden=2, n_neurons=300, learning_rate=3e-3, input_shape= (784,)):
 model = keras.models.Sequential()
 options = {"input_shape": input_shape}
 for layer in range(n_hidden):
     model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
     options = {}
 model.add(keras.layers.Dense(10, activation= 'softmax', **options))
 optimizer = keras.optimizers.SGD(learning_rate)
 model.compile(loss='sparse_categorical_crossentropy', 
             optimizer= optimizer, metrics=['accuracy'])
 return model

In [36]:
keras_class = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [37]:
checkpoint_cb= keras.callbacks.ModelCheckpoint('2fashion_mnist_model.h5', save_best_only=True)

In [ ]:
keras_class.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])
mse_test = keras_class.score(X_test, y_test)

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 442s 8ms/sample - loss: 0.9958 - acc: 0.6934 - val_loss: 0.6672 - val_acc: 0.7864
Epoch 2/100
55000/55000 [==============================] - 43s 774us/sample - loss: 0.6172 - acc: 0.7944 - val_loss: 0.5559 - val_acc: 0.8164
Epoch 3/100
55000/55000 [==============================] - 47s 852us/sample - loss: 0.5404 - acc: 0.8167 - val_loss: 0.5013 - val_acc: 0.8358
Epoch 4/100
55000/55000 [==============================] - 2593s 47ms/sample - loss: 0.5013 - acc: 0.8272 - val_loss: 0.4786 - val_acc: 0.8402
Epoch 5/100
55000/55000 [==============================] - 50s 900us/sample - loss: 0.4750 - acc: 0.8358 - val_loss: 0.4566 - val_acc: 0.8462
Epoch 6/100
55000/55000 [==============================] - 39s 707us/sample - loss: 0.4569 - acc: 0.8411 - val_loss: 0.4411 - val_acc: 0.8536
Epoch 7/100
55000/55000 [==============================] - 40s 735us/sample - loss: 0.4419 - acc: 0

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
 "n_hidden": [3], "n_neurons": [100], "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_class, param_distribs, n_iter=1, cv=3, scoring='accuracy')
rnd_search_cv.fit(X_train, y_train, epochs=2,validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
print('done')

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
new_model2 = rnd_search_cv.best_estimator_.keras_class

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
 "n_hidden": [0, 1, 2, 3], "n_neurons": np.arange(1, 100), "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_class, param_distribs, n_iter=10, cv=3,
                 scoring='accuracy')
rnd_search_cv.fit(X_train, y_train, epochs=100,validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])